#### Restart & Run All Then Call Ord-Log

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 11, 5)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,24,S,SCCC,"1,200",149.50,1,CUT,SET,2022-08-13


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade     n...)

In [4]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [5]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BANPU,7500,12.00,1,XXX,SET50,2022-09-14
1,3,B,BLA,4000,31.00,0,1L,SET50,2022-08-23
2,5,B,JMART,3000,44.00,1,1L,SET50,2022-04-19
3,6,B,JMT,1200,63.00,1,1L,SET50,2022-04-19
4,7,B,KCE,1000,41.00,2,5pct,SET50,2022-08-22
5,10,B,PTTGC,3000,41.25,2,5pct,SET50,2022-09-02
6,12,B,SCB,1000,102.00,0,XXX,SET50,2022-02-14
7,20,S,IVL,2400,44.25,1,5%,SET50,2022-08-30
8,23,S,SCC,300,340.00,2,C398,SET50,2022-08-10


In [6]:
name = 'JMT'
limit = 20 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,20.00,20.00,20.00,20.00,20.00
mean,64.45,65.38,63.68,7674706.50,64.61
std,1.93,1.66,2.12,3541704.24,1.99
min,59.00,61.25,58.50,2951580.00,59.00
25%,63.69,64.75,63.19,5044061.75,64.19
50%,64.88,65.75,64.25,6741567.00,65.00
75%,65.31,66.50,64.81,9933574.50,65.62
max,66.75,67.75,66.25,14052352.00,67.25


#### 2) Set100 records

In [7]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,BCH,3000,17.5,2,C21.46,SET100,2022-08-31
1,11,B,RCL,3000,27.0,2,C27.75,SET100,2022-08-18
2,13,B,SINGER,3000,35.5,1,52WL,SET100,2022-02-02
3,14,B,STA,2500,18.1,2,5pct,SET100,2022-08-23
4,16,B,SYNEX,3000,15.2,2,5pct,SET100,2022-08-22
5,18,S,DOHOME,2800,13.6,1,RD20pct,SET100,2022-02-02


In [8]:
name = 'BAM'
limit = 60 # 3 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,16.95,17.14,16.77,14781448.95,16.98
std,1.31,1.28,1.30,9163408.89,1.28
min,14.80,14.90,14.50,4242783.00,14.80
25%,15.58,15.70,15.30,9299233.25,15.62
50%,17.35,17.60,17.15,13007340.00,17.35
75%,18.20,18.40,18.10,17023462.75,18.20
max,18.60,18.60,18.30,60098963.00,18.50


#### 3) Set records

In [9]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,CPNREIT,10000,18.2,1,HD,SET,2022-08-17
1,8,B,KSL,30000,3.3,0,6L,SET,2022-02-02
2,9,B,LANNA,3000,18.0,1,3L,SET,2023-03-18
3,15,B,STARK,30000,3.3,1,XXX,SET,2022-02-02
4,17,S,ASP,10000,3.1,1,5pct,SET,2022-08-24
5,19,S,GVREIT,10000,9.1,1,C8.90,SET,2022-08-26
6,21,S,JASIF,10000,8.5,1,CP1S,SET,2022-08-19
7,22,S,MCS,25000,9.7,2,5pct,SET,2022-09-03
8,24,S,SCCC,1200,149.5,1,CUT,SET,2022-08-13


In [10]:
name = 'LANNA'
limit = 120 # 6 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,20.63,20.93,20.35,4293385.79,20.65
std,1.72,1.74,1.65,4174595.20,1.68
min,17.80,18.40,17.80,793597.00,18.20
25%,19.30,19.50,19.10,2042389.75,19.38
50%,20.20,20.40,19.80,3122115.00,20.10
75%,21.95,22.42,21.52,4368007.00,22.02
max,24.40,24.60,23.90,27453276.00,24.30


In [11]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BANPU,7500,12.00,1,XXX,SET50,2022-09-14
1,2,B,BCH,3000,17.50,2,C21.46,SET100,2022-08-31
2,3,B,BLA,4000,31.00,0,1L,SET50,2022-08-23
3,4,B,CPNREIT,10000,18.20,1,HD,SET,2022-08-17
4,5,B,JMART,3000,44.00,1,1L,SET50,2022-04-19
5,6,B,JMT,1200,63.00,1,1L,SET50,2022-04-19
6,7,B,KCE,1000,41.00,2,5pct,SET50,2022-08-22
7,8,B,KSL,30000,3.30,0,6L,SET,2022-02-02
8,9,B,LANNA,3000,18.00,1,3L,SET,2023-03-18
9,10,B,PTTGC,3000,41.25,2,5pct,SET50,2022-09-02
